# Projeto 2 - Ciência dos Dados

Nome: _____

Nome: _____

Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [116]:
%%capture

#Instalando o tweepy
!pip install tweepy

In [117]:
import tweepy
import math
import os.path
import pandas as pd
import json
from random import shuffle

___
## Autenticando no  Twitter

* Conta: @EikiMarcelo

In [118]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @EikiMarcelo

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [119]:
#Produto escolhido:
produto = 'Skol'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [120]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():    
    msgs.append(msg.full_text.lower())
    i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

In [121]:
msgs

['@hoteldeioona beber corotes e skol beats juntos',
 'rt @santtanatlg: ☀️barraca point por do sol ☀️ \n\n❌localizada na praia da porteira/manguinho \n\n🔆temos:\n• porções (pastel, isca de peixe) \n•…',
 'meus amigos são aleatórios, fui p tonheiros, depois fui pra faculdade, sai de novo, fui na pague menos e voltei pro tonheiros e ainda encontrei mais gente, e agora to na casa dos meninos tomando skol e ouvindo raul seixas, em plena segunda?hahaha 😍 meus amigos são meus bebês 😍',
 'agora será eu e os meus meninos 💙 isma, benjamin e o skol 💙 como diz o meu marido sou a rainha da casa 👸🏾',
 'ñ podia faltar skol beats e amigas \n\ndedinho de gabs https://t.co/x84lmbuu0t',
 '[9/9 20:13] : tem cerveja aqui em casa\n[9/9 20:18] ju: skol beats\n[9/9 20:18] : vou descer antes pra comprar\n[9/9 20:18] : compro então? a azul?\n\nnamoral que inferno de pessoa perfeita 😍😍😍😍😍😍',
 'tava tão desesperada pelo trabalho de artes ser amanhã que eu peguei as únicas latas que tinham que era red bull e skol 

Salvando os dados em uma planilha Excel:

In [122]:
len (msgs)

500

In [123]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto), engine = 'openpyxl')

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
### Classificando as mensagens na coragem

Esta etapa é manual. Faça a mesma pelo Excel.

In [124]:
#def de limpar palavras
import re 

def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    punctuation = '[!\-.:?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed
#Excel
skol_read=pd.read_excel("skol.xlsx")

In [157]:
#Limpeza e separação por relevancia
skol_read.Treinamento.apply(cleanup)
skol_read.rename(columns={'Unnamed: 1':'relevante'},inplace=True)
#Relevante 0
skol_relevante_0=skol_read[skol_read.relevante==0]

#Relevante 1
skol_relevante_1=skol_read[skol_read.relevante==1]

In [160]:
# Separando as palavras Relevancia 1
"".join(skol_relevante_1["Treinamento"]).split()
palavras=pd.DataFrame("".join(skol_relevante_1["Treinamento"]).split())
palavras[0].value_counts()
palavras_relevancia_1=palavras[0].value_counts()

skol               97
de                 51
e                  36
beats              35
que                34
a                  31
uma                26
é                  25
pra                24
eu                 17
não                15
o                  15
q                  15
ela                12
na                 11
com                10
mais               10
um                  9
da                  9
mas                 8
do                  8
me                  8
se                  8
em                  7
tomar               7
vontade             7
cerveja             7
fazer               6
mano                6
vai                 6
                   ..
deusssssssgente     1
aproveita           1
compras             1
1-                  1
estados             1
camarote            1
ficar               1
saúde               1
vinho               1
vida,               1
carro*              1
buraco              1
viu                 1
skol..              1
gosto     

In [162]:
# Separando as palavras Relevancia 0
"".join(skol_relevante_0["Treinamento"]).split()
palavras=pd.DataFrame("".join(skol_relevante_0["Treinamento"]).split())
palavras[0].value_counts()
palavras_relevancia_0=palavras[0].value_counts()
palavras_relevancia_0

skol                                     115
de                                        98
e                                         67
a                                         63
beats                                     41
o                                         38
eu                                        37
da                                        35
que                                       33
pra                                       30
uma                                       29
é                                         27
do                                        27
me                                        23
com                                       22
no                                        20
q                                         18
não                                       18
em                                        17
na                                        16
um                                        16
se                                        16
tu        

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**